# Redis Sorted Sets Tutorial

## Setup

In [ ]:
%%capture
%pip install redis rich tqdm 

In [ ]:
import redis
from rich.pretty import pprint

In [ ]:
# creating cluster
# make sure jupyter server is connected to redis network
# `docker network connect redis_default jupyter-jupyter-1`
r = redis.RedisCluster(host='master', port=6379)

## Redis Sorted Sets

Redis sets are unsorted, which limits their usefulness. Sorted sets are similar to sets, except each unique member has an associated score, which provides the mechanism for sorting. Use cases include:

- rate limiters
- game leaderboards

The following example creates a set of bike brands named for famous computer programmers, using their birth year as the score element to sort the set.

In [ ]:
# Create a sorted set
r.zadd("bike:brands", {"Alan Kay": 1940})
r.zadd("bike:brands", {"Grace Hopper": 1906})
r.zadd("bike:brands", {"Richard Stallman": 1953})
r.zadd("bike:brands", {"Yukihiro Matsumoto": 1965})
r.zadd("bike:brands", {"Claude Shannon": 1916})
r.zadd("bike:brands", {"Linus Torvalds": 1969})
r.zadd("bike:brands", {"Sophie Wilson": 1957})
r.zadd("bike:brands", {"Alan Turing": 1912})

print("Created sorted set of bike brands with birth years as scores")

Now you can use the `ZRANGE` command to retrieve members in the order of birth year using member ranks as the arguments. As with lists, you can use `-1`, `-2`, etc., to represent the last, second to last, etc., members.

In [ ]:
# ZRANGE usage
result = r.zrange("bike:brands", 2, 4)
print("Members from rank 2 to 4:")
for item in result:
    print(f"  - {item}")

To delete members from a sorted set, use the `ZREM` command.

In [ ]:
# Remove the Alan Turing model and score
removed = r.zrem("bike:brands", "Alan Turing")
print(f"Removed Alan Turing: {removed == 1}")

Once all the members of a sorted set have been removed, the set's key will also be removed.

To list all the members in score order, use the `ZRANGEBYSCORE`. If you pass the `WITHSCORES` argument, each member's score will also be listed.

In [ ]:
# List members ordered by score
result = r.zrangebyscore("bike:brands", "-inf", "+inf", withscores=True)
print("All bike brands ordered by birth year:")
for member, score in result:
    print(f"  - {member}: {int(score)}")

Note the use of `-inf` and `+inf` in the previous example. This is useful when you don't readily know the score range or particular values. See the manual page for `ZRANGEBYSCORE` for more options.

You can get the rank of any sorted set member using the `ZRANK` command. Note: the member with the lowest score has rank `0`.

In [ ]:
# Get a bike brand's rank
rank = r.zrank("bike:brands", "Claude Shannon")
score = r.zscore("bike:brands", "Claude Shannon")
print(f"Claude Shannon rank: {rank}")
print(f"Claude Shannon score (birth year): {int(score)}")

## Resources

- Sorted set type [reference page](https://redis.io/docs/data-types/sorted-sets?utm_source=redisinsight&utm_medium=main&utm_campaign=tutorials).
- Entire list of [Redis sorted set commands](https://redis.io/commands/?group=sorted-set&utm_source=redisinsight&utm_medium=main&utm_campaign=tutorials).
- Check out [Get started with Redis](https://university.redis.io/learningpath/14q8m6gilfwltm?utm_source=redisinsight&utm_medium=main&utm_campaign=tutorials) learning path on Redis University for an introduction to working with all core data structures in Redis.